Instalar las librerias necesarias

In [3]:
%pip install mysql-connector-python
%pip install fastapi
%pip install pyngrok
%pip install uvicorn
%pip install python-multipart

  Using cached mysql_connector_python-9.2.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (6.0 kB)
Using cached mysql_connector_python-9.2.0-cp311-cp311-manylinux_2_28_x86_64.whl (34.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.7 MB/s eta 0:00:00


Importar los modulos necesarios de las librerias que se instalaron.

In [4]:
import pandas as pd
import mysql.connector
import nest_asyncio
import uvicorn
import io
import logging
from pyngrok import ngrok
from fastapi import FastAPI, File, UploadFile, Form, HTTPException

In [5]:
!ngrok config add-authtoken 2tYc54ffpS0VN3DuNgup3ItwTem_2vgeRnWXzN8Tb7RMYsHvB

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Funcion que se conecta a la base de datos y corre los comandos de creacion de tablas
# con los parametros que se reciben mediante csv.
def create_tables():
  conn = mysql.connector.connect(**db_connection)
  cursor = conn.cursor()
  cursor.execute("CREATE TABLE jobs (id INT PRIMARY KEY, job VARCHAR(255));")
  cursor.execute("CREATE TABLE departments (id INT PRIMARY KEY, department VARCHAR(255));")
  cursor.execute("CREATE TABLE hired_employees (id INT PRIMARY KEY, name VARCHAR(255), datetime VARCHAR(255), department_id INT, job_id INT);")
  cursor.close()
  conn.close()

db_connection = {
    "database": "b6lzfmzs2xs2xqr2zf5r",
    "user": "uatmmjtzox50q4hw",
    "password": "aET1zCwQKBCwkBImyTaG",
    "host": "b6lzfmzs2xs2xqr2zf5r-mysql.services.clever-cloud.com",
    "port": 3306
}

create_tables()


In [6]:
# Se llenan las tablas con la informacion que se recibe de los csv y se conectan
# a la base de datos para escribir en las tablas.
def fill_tables(csv_df, table):
  conn = mysql.connector.connect(**db_connection)
  cursor = conn.cursor()
  df = pd.read_csv(csv_df)
  for _, row in df.iterrows():
    row_str = ','.join(['%s'] * len(row))
    cursor.execute(f"INSERT INTO {table} VALUES ({row_str});", tuple(row))
  cursor.close()
  conn.close()

# Se leen los archivos csv y se corre la funcion con el parametro y con el nombre
# de la tabla.
jobs = '/content/jobs.csv'
departments = '/content/departments.csv'
hired_employees = '/content/hired_employees.csv'
fill_tables(jobs, 'jobs')
fill_tables(departments, 'departments')
fill_tables(hired_employees, 'hired_employees')

In [ ]:
# Se crea la API con FastAPI que permite hacer un post para insertar la data. La
# data que se puede insertar se realiza mediante un CSV que se lee con el metodo
# post y que transforma para hacer la insercion en la tabla.
app = FastAPI()

@app.post("/insert_data/")
async def insert_data(csv: UploadFile = File(...)):
  conn = mysql.connector.connect(**db_connection)
  cursor = conn.cursor()
  print(csv.filename)
  contents = await csv.read()

  df = pd.read_csv(io.StringIO(contents.decode("utf-8",errors="ignore")))

  for _, row in df.iterrows():
    row_str = ','.join(map(str, row.values))
    cursor.execute(f"INSERT INTO {csv.filename} VALUES ({row_str});")
  return "Exitoso"

if __name__ == "__main__":
  nest_asyncio.apply()
  ngrok.set_auth_token("2tYc54ffpS0VN3DuNgup3ItwTem_2vgeRnWXzN8Tb7RMYsHvB")
  print(ngrok.connect(8000).public_url)
  uvicorn.run(app, host="0.0.0.0",port=8000)